In [1]:
import pandas as pd

In [5]:
def split(x, i):
    a=x.split()
    if i>0:
        return int(a[i])
    else: 
        return a[i]

def traitement_annotation(file):
    
    data = pd.read_csv(file, sep='\t', header=None)
    data.columns=['T', 'a_split', 'valeur']
    
    data['classe']=data["a_split"].apply(lambda x: split(x, 0))
    data['debut']=data["a_split"].apply(lambda x: split(x, 1))
    data['fin']=data["a_split"].apply(lambda x: split(x, 2))
    
    data=data.drop(columns=['a_split'])
    data=data.reindex(columns=['T', 'classe', 'valeur', 'debut', 'fin'])
    data=data.sort_values('debut').reset_index(drop=True)
    
    a_supprimer=[0]
    for i in range(1,data.shape[0]):
        if data.loc[i,'classe']==data.loc[i-1,'classe'] and data.loc[i-1,'fin']==data.loc[i,'debut']:
            data.loc[i,'valeur']=data.loc[i-1,'valeur']+data.loc[i,'valeur']
            data.loc[i,'debut']= data.loc[i-1,'debut']
            a_supprimer[-1]=1
            a_supprimer.append(0)
        else:
            a_supprimer.append(0)

    data['a_supprimer']=a_supprimer
    
    data=data[data.a_supprimer==0]
    data=data.drop(columns=['a_supprimer'])
    data=data.reset_index(drop=True)
    
    return data

def links_dataframe(df):
    relations= pd.DataFrame(columns=['classe_source','valeur_source', 'classe_dest', 'valeur_dest'])
    classe_source=[]
    valeur_source=[]
    classe_dest=[]
    valeur_dest=[]
    
    for i in range (df.shape[0]-1):
        if df.loc[i, 'classe']=='examen' and df.loc[i+1,'classe']=='valeur' and df.loc[i+1,'debut']-df.loc[i,'fin']<100:
            classe_source.append('examen')
            valeur_source.append(df.loc[i,'valeur'])
            classe_dest.append('valeur')
            valeur_dest.append(df.loc[i+1,'valeur'])
            
    relations['classe_source'],relations['valeur_source'], relations['classe_dest'], relations['valeur_dest']=classe_source,valeur_source,classe_dest, valeur_dest
    return relations

In [3]:
data = traitement_annotation("output/filepdf-838-cas.ann")
data

,T,classe,valeur,debut,fin
0,T9,sosy,masse palpébrale gauche,126,149
1,T3,examen,examen ophtalmologique,183,205
2,T10,sosy,élévation du globe oculaire,520,547
3,T2,valeur,normal,576,582
4,T4,examen,examen de l’,652,664
5,T5,valeur,sans particularité,682,700
6,T6,examen,examen général,705,719
7,T8,examen,examen n’,837,846
8,T7,examen,TDM,887,890
9,T1,valeur,stade 3,1085,1092


In [6]:
links_dataframe(data)

,classe_source,valeur_source,classe_dest,valeur_dest
0,examen,examen de l’,valeur,sans particularité
